# Experiment 004: Structured Grid Packing

This experiment implements structured packing using grid translations:
1. Use 2 base trees with optimal rotation angles
2. Translate them in a grid pattern (nx × ny where nx*ny >= N)
3. Optimize the translation distances (dx, dy) to minimize bounding box
4. This creates collision-free packings BY CONSTRUCTION

In [ ]:
import numpy as np
import pandas as pd
import os
from numba import njit
import math
import time
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import shutil
from itertools import product

os.chdir('/home/code/experiments/004_structured_packing')
print(f'Working directory: {os.getcwd()}')

In [ ]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def get_bbox(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    side = get_bbox(xs, ys, degs, tx, ty)
    return side * side / n

tx, ty = make_polygon_template()
print('Template created')

In [ ]:
# Load baseline submission
shutil.copy('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv', 'submission.csv')

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        configs[n] = {'x': xs, 'y': ys, 'deg': ds}
    return configs

configs = load_submission('submission.csv')

# Calculate initial score
initial_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        initial_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')

In [ ]:
# Overlap detection
def get_shapely_polygon(cx, cy, deg, tx, ty):
    r = deg * np.pi / 180.0
    c = np.cos(r)
    s = np.sin(r)
    px = c * tx - s * ty + cx
    py = s * tx + c * ty + cy
    return Polygon(zip(px, py))

def has_overlap(xs, ys, degs, tx, ty):
    n = len(xs)
    if n <= 1:
        return False
    
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print('Overlap detection defined')

In [ ]:
# Structured grid packing
def structured_packing(n, tx, ty, verbose=False):
    """Create N trees using grid translation pattern."""
    best_score = float('inf')
    best_config = None
    
    # Try different grid dimensions
    grid_dims = []
    for nx in range(1, n+1):
        ny = (n + nx - 1) // nx
        if nx * ny >= n:
            grid_dims.append((nx, ny))
    
    # Angle combinations to try
    angles = [0, 30, 45, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
    
    # Translation distances to try
    dx_range = np.linspace(0.4, 1.5, 25)
    dy_range = np.linspace(0.4, 1.5, 25)
    
    for nx, ny in grid_dims:
        for angle1 in angles:
            for angle2 in angles:
                for dx in dx_range:
                    for dy in dy_range:
                        # Create grid of trees
                        xs = []
                        ys = []
                        degs = []
                        
                        count = 0
                        for i in range(nx):
                            for j in range(ny):
                                if count >= n:
                                    break
                                x = i * dx
                                y = j * dy
                                angle = angle1 if (i + j) % 2 == 0 else angle2
                                xs.append(x)
                                ys.append(y)
                                degs.append(angle)
                                count += 1
                            if count >= n:
                                break
                        
                        xs = np.array(xs)
                        ys = np.array(ys)
                        degs = np.array(degs)
                        
                        # Check for overlaps
                        if has_overlap(xs, ys, degs, tx, ty):
                            continue
                        
                        # Calculate score
                        score = score_group(xs, ys, degs, tx, ty)
                        
                        if score < best_score:
                            best_score = score
                            best_config = {'x': xs.copy(), 'y': ys.copy(), 'deg': degs.copy()}
                            if verbose:
                                print(f'  N={n}: Found better config with score {score:.6f} (grid {nx}x{ny}, angles {angle1}/{angle2}, dx={dx:.2f}, dy={dy:.2f})')
    
    return best_config, best_score

print('Structured packing function defined')

In [ ]:
# Test structured packing on a few N values
print('Testing structured packing on selected N values...')
start_time = time.time()

improved_configs = {}

# Focus on N values that might benefit from structured packing
# Start with small N values (1-10) to test
test_n_values = list(range(1, 11))

for n in test_n_values:
    if n not in configs:
        continue
    
    c = configs[n]
    baseline_score = score_group(c['x'], c['y'], c['deg'], tx, ty)
    
    # Try structured packing
    new_config, new_score = structured_packing(n, tx, ty, verbose=False)
    
    if new_config is not None and new_score < baseline_score - 1e-10:
        print(f'N={n}: {baseline_score:.8f} -> {new_score:.8f} (improved by {baseline_score - new_score:.8f})')
        improved_configs[n] = new_config
    else:
        print(f'N={n}: {baseline_score:.8f} (no improvement, structured={new_score:.8f} if found)')

print(f'\nTime: {time.time() - start_time:.1f}s')
print(f'Improved {len(improved_configs)} configurations')

In [ ]:
# Try larger N values (where crystalline packing should help)
print('\nTesting structured packing on larger N values...')
start_time = time.time()

# Focus on perfect squares and near-perfect rectangles
large_n_values = [16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196]

for n in large_n_values:
    if n not in configs:
        continue
    
    c = configs[n]
    baseline_score = score_group(c['x'], c['y'], c['deg'], tx, ty)
    
    # Try structured packing
    new_config, new_score = structured_packing(n, tx, ty, verbose=False)
    
    if new_config is not None and new_score < baseline_score - 1e-10:
        print(f'N={n}: {baseline_score:.8f} -> {new_score:.8f} (improved by {baseline_score - new_score:.8f})')
        improved_configs[n] = new_config
    else:
        print(f'N={n}: {baseline_score:.8f} (no improvement)')

print(f'\nTime: {time.time() - start_time:.1f}s')
print(f'Total improved: {len(improved_configs)} configurations')

In [ ]:
# Update configs with improvements
for n, new_config in improved_configs.items():
    configs[n] = new_config

# Calculate new total score
new_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        new_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')
print(f'New score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')

In [ ]:
# Validate for overlaps
print('Validating for overlaps...')
overlap_n = []
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        if has_overlap(c['x'], c['y'], c['deg'], tx, ty):
            overlap_n.append(n)

if overlap_n:
    print(f'Overlaps found in N: {overlap_n}')
else:
    print('No overlaps detected')

In [ ]:
# Save submission
def save_submission(configs, filepath):
    rows = []
    for n in range(1, 201):
        if n in configs:
            c = configs[n]
            for i in range(len(c['x'])):
                rows.append({
                    'id': f'{n:03d}_{i}',
                    'x': f's{c["x"][i]}',
                    'y': f's{c["y"][i]}',
                    'deg': f's{c["deg"][i]}'
                })
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f'Saved to {filepath}')

save_submission(configs, 'submission.csv')
shutil.copy('submission.csv', '/home/submission/submission.csv')
print('Submission saved to /home/submission/submission.csv')

In [ ]:
# Final summary
print(f'\n=== EXPERIMENT 004 SUMMARY ===')
print(f'Initial score: {initial_score:.6f}')
print(f'Final score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')
print(f'Overlaps: {len(overlap_n)}')
print(f'Improved N values: {list(improved_configs.keys())}')